In [ ]:
import os

import torch
import torchvision
from torch import nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import MNIST
from torchvision.utils import save_image
from torch.utils.data import Dataset, DataLoader
import torchvision.datasets as datasets
import matplotlib.pyplot as plt 

In [ ]:
class autoencoder(nn.Module):
    def __init__(self):
        super(autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(28 * 28, 128),
            nn.ReLU(True),
            nn.Linear(128, 64),
            nn.ReLU(True), 
            nn.Linear(64, 12), 
            nn.ReLU(True), 
            nn.Linear(12, 3))
        self.decoder = nn.Sequential(
            nn.Linear(3, 12),
            nn.ReLU(True),
            nn.Linear(12, 64),
            nn.ReLU(True),
            nn.Linear(64, 128),
            nn.ReLU(True), 
            nn.Linear(128, 28 * 28), 
            nn.Tanh())

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

In [ ]:
mnist_trans = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(0, 1.0),
    transforms.CenterCrop((28))
])

In [ ]:
num_epochs = 100
batch_size = 128
learning_rate = 1e-3

In [ ]:
mnist= datasets.MNIST(root='./../data', train=True,download=False, transform=mnist_trans)
mnist = torch.utils.data.random_split(mnist, [10000, len(mnist)-10000])[0] # gets split into two parts [10k and 50k] and we slice 0 index.

dataloader = DataLoader(mnist,batch_size=batch_size)

In [ ]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

print('Using PyTorch version:', torch.__version__, ' Device:', device)

torch.set_default_dtype(torch.float64)
torch.set_default_device(device)

In [ ]:
model = autoencoder().cuda()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-5)

for epoch in range(num_epochs):
    for data in dataloader:

        img, _ = data
        img = img.view(img.size(0), -1)
        
        output = model(img.cuda())
        loss = criterion(output, img.cuda())
        

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # ===================log========================
    print('epoch [{}/{}], loss:{:.4f}'
          .format(epoch + 1, num_epochs, loss))
    if epoch % 1 == 0:

        pred_img = mnist[0][0].view(mnist[0][0].size(0), -1)
        prediction = model(pred_img.cuda()).detach().cpu().numpy()
        print(prediction.shape)
        plt.imshow(prediction.reshape([28,28]),cmap='gray')
        plt.show()
        # save_image(pic, './mlp_img/image_{}.png'.format(epoch))

torch.save(model.state_dict(), './sim_autoencoder.pth')